## Problem statement
--- 
Day 11 is a graph/network problem, the inputs detail a list of nodes and their one-way directed connections. Part 1 asks us to find the number of paths that:
1. Start from `'you'`
2. End in `'out'`

Let's load the data, the use of ':' in the input data signals a good use case for dictionaries using that as the splitting character.

In [16]:
d = {}
with open('puzzle_input.txt', 'r') as f:
    for line in f:
        (key, val) = line.split(':')
        val = val.rstrip()[1:].split(' ')
        d[key] = val


## Part 1
---

Part 1 is pretty straight forward to solve with the networkx library, you simply create a graph containing all the nodes and add the edges as instructed by the input file. The number of paths from 'you' to 'out' can be counted via `nx.all_simple_paths`.

See below how the sample network looks like.

![](sample_network.png)


In [22]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()

for key, val in d.items():
    for node in val:
        G.add_edge(key, node)

paths = nx.all_simple_paths(G, source='you', target='out')

count = 0
for path in paths:
    count += 1

print(count)

413


## Part 2
---
Part 2 asks us to find the number of paths that go from 'svr' to 'out' but that also visit 'fft' and 'dac' (it must be in this order as there are none that go from 'fft' to 'dac' later on). 

This cannot be done with the same method as enumerating all of the paths is a very large search space, look below for a visualisation (using Grephi) where:

- 'svr' is red
- 'out' is teal
- 'fft' is green
- 'dac' is blue


<img src="part2.png" alt="drawing" width="600"/>

After some research, this is what's called a DAG (directed acyclic graph) path count problem and it must be done through dynamic programming as listing all the paths meeting this criteria is not tractable (it grows exponentially). However, counterintuively, counting the number of paths is tractable as memoization allows the counts to happen in $O(V + E)$ where V is the number of vertices and E the number of edges.

In [39]:
def count_paths(source: str, target: str) -> int:
    ## Create empty dictionary for memoization
    d = {}
    
    def path_counter(curr: str, target: str) -> int:

        #Case we arrive at the target node
        if curr == target:
            d[curr] = 1
            return d[curr]

        #Case we arrive at dead-end node
        if len(G.edges(curr)) == 0:
            d[curr] = 0
            return d[curr]

        #Case we need to figure out how many paths there are to target based on edges to other nodes
        count = 0
        for edge in G.edges(curr):
            if edge[1] in d:
                count += d[edge[1]]

            else:
                count += path_counter(edge[1], target)

        d[curr] = count
        return count
            
    return path_counter(source, target)

'''
Must be done this way as memoization loses information about the path sequences so we must first find the number of paths to the first stop
'svr' -> 'fft', then
'fft' -> 'dac', then
'dac' -> 'out'

The solution is then the multiple of all of these as there are X many ways from svr to fft, then Y many ways frmo fft to dac then Z many ways from dac to out.
'''
count1 = count_paths('svr', 'fft')
count2 = count_paths('fft', 'dac')
count3 = count_paths('dac', 'out')

print(f"Paths from svr -> fft = {count1}, fft -> dac = {count2}, dac -> out = {count3}")
print(count1*count2*count3)

Paths from svr -> fft = 5392, fft -> dac = 9991035, dac -> out = 9755
525518050323600
